In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.models import Model

print(tf.__version__)
print(np.__version__)

2.6.0
1.21.4


In [53]:
# function for building resnet Block

# build_residual_block(x, 64, name=f'stage2_{i}', is_50=is_50)

def build_residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False, name=None, is_50=False):
    shortcut = x

    if conv_shortcut:
        shortcut = Conv2D(filters, 1, strides=stride, name=name + '_short')(x)
        shortcut = BatchNormalization(name=name + '_bn4')(shortcut)
    
    if is_50:
        x = Conv2D(filters, kernel_size=1, padding='same', strides=stride, name=name + '_conv1')(x)
        x = BatchNormalization(name=name + '_bn1')(x)
        x = Activation('relu', name=name + '_relu1')(x)

    x = Conv2D(filters, kernel_size, padding='same', strides=stride, name=name + '_conv2')(x)
    x = BatchNormalization(name=name + '_bn2')(x)
    x = Activation('relu', name=name + '_relu2')(x)
    
    if is_50:
#         x = Conv2D(filters*4, kernel_size=1, padding='same', name=name + '_conv3')(x)
        x = Conv2D(filters*4, kernel_size=1, name=name + '_conv3')(x)
        x = BatchNormalization(name=name + '_bn3')(x)
    else:
        x = Conv2D(filters, kernel_size, padding='same', name=name + '_conv3')(x)
        x = BatchNormalization(name=name + '_bn3')(x)

    x = Add(name=name + '_add')([x, shortcut])

    x = Activation('relu', name=name + '_relu4')(x)
    
    return x

In [48]:
# ResNet 모델 자체를 생성하는 함수입니다.
def build_resnet(input_shape, num_classes, is_50):
    inputs = Input(shape=input_shape, name='input_layer')

    x = Conv2D(64, 7, strides=2, padding='same', name='conv2d_15')(inputs)
    x = BatchNormalization(name='batch_normalization_13')(x)
    x = Activation('relu', name='activation_568')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='stage2_0_maxpooling')(x)

    # Stage 2
    for i in range(1, 4):
        x = build_residual_block(x, 64, name=f'stage2_{i}', is_50=is_50)

    # Stage 3
    x = build_residual_block(x, 128, stride=2, conv_shortcut=True, name='stage3_1', is_50=is_50)
    for i in range(2, 5):
        x = build_residual_block(x, 128, name=f'stage3_{i}', is_50=is_50)

    # Stage 4
    x = build_residual_block(x, 256, stride=2, conv_shortcut=True, name='stage4_1', is_50=is_50)
    for i in range(2, 7):
        x = build_residual_block(x, 256, name=f'stage4_{i}', is_50=is_50)

    # Stage 5
    x = build_residual_block(x, 512, stride=2, conv_shortcut=True, name='stage5_1', is_50=is_50)
    for i in range(2, 4):
        x = build_residual_block(x, 512, name=f'stage5_{i}', is_50=is_50)
    
    if is_50:
        x = AveragePooling2D(pool_size=(7, 7), name='avg_pool')(x)
    else:
        x = AveragePooling2D(pool_size=(1, 1), name='avg_pool')(x)
    x = Flatten(name='flatten_11')(x)
    outputs = Dense(num_classes, activation='softmax', name='fc1000')(x)

    model = Model(inputs, outputs, name='resnet34')
    return model

In [49]:
"""
Total params: 21,315,338
Trainable params: 21,298,314
Non-trainable params: 17,024
"""
input_shape = (32, 32, 3)
num_classes = 10
resnet_34 = build_resnet(input_shape, num_classes, is_50=False)
resnet_34.summary()


Model: "resnet34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 16, 16, 64)   9472        input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 16, 16, 64)   256         conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_568 (Activation)     (None, 16, 16, 64)   0           batch_normalization_13[0][0]     
___________________________________________________________________________________________

In [50]:
input_shape = (32, 32, 3)
num_classes = 10
resnet_50 = build_resnet(input_shape, num_classes, is_50=True)
resnet_50.summary()


ValueError: Operands could not be broadcast together with shapes (7, 7, 256) (7, 7, 64)

In [69]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model


def build_residual_block(input_layer, filters, kernel_size, stage, block):
    conv_name_base = f'res{stage}_{block}_branch'
    bn_name_base = f'bn{stage}_{block}_branch'

    x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same', name=conv_name_base + '_2a')(input_layer)
    x = BatchNormalization(axis=3, name=bn_name_base + '_2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same', name=conv_name_base + '_2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '_2b')(x)

    shortcut = Conv2D(filters=filters, kernel_size=(1, 1), padding='valid', name=conv_name_base + '_1')(input_layer)
    shortcut = BatchNormalization(axis=3, name=bn_name_base + '_1')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x


def build_resnet(input_shape=(32, 32, 3), is_50=True, num_classes=10):
    input_layer = Input(shape=input_shape, name='input_layer')

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), name='conv1_conv')(input_layer)
    x = BatchNormalization(axis=3, name='conv1_bn')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='pool1_pool')(x)

    # Choose between ResNet50 or ResNet34 based on is_50 flag
    if is_50:
        x = build_residual_block(x, filters=64, kernel_size=3, stage=2, block='a')
        x = build_residual_block(x, filters=64, kernel_size=3, stage=2, block='b')
        x = build_residual_block(x, filters=64, kernel_size=3, stage=2, block='c')
        x = build_residual_block(x, filters=128, kernel_size=3, stage=3, block='a')
        x = build_residual_block(x, filters=128, kernel_size=3, stage=3, block='b')
        x = build_residual_block(x, filters=128, kernel_size=3, stage=3, block='c')
        x = build_residual_block(x, filters=128, kernel_size=3, stage=3, block='d')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='a')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='b')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='c')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='d')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='e')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='f')
        x = build_residual_block(x, filters=512, kernel_size=3, stage=5, block='a')
        x = build_residual_block(x, filters=512, kernel_size=3, stage=5, block='b')
        x = build_residual_block(x, filters=512, kernel_size=3, stage=5, block='c')
    else:
        x = build_residual_block(x, filters=64, kernel_size=3, stage=2, block='a')
        x = build_residual_block(x, filters=64, kernel_size=3, stage=2, block='b')
        x = build_residual_block(x, filters=128, kernel_size=3, stage=3, block='a')
        x = build_residual_block(x, filters=128, kernel_size=3, stage=3, block='b')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='a')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='b')
        x = build_residual_block(x, filters=256, kernel_size=3, stage=4, block='c')
        x = build_residual_block(x, filters=512, kernel_size=3, stage=5, block='a')
        x = build_residual_block(x, filters=512, kernel_size=3, stage=5, block='b')
        x = build_residual_block(x, filters=512, kernel_size=3, stage=5, block='c')

    x = AveragePooling2D(pool_size=(2, 2), padding='same', name='avg_pool')(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax', name='fc')(x)

    model = Model(inputs=input_layer, outputs=x, name='ResNet50' if is_50 else 'ResNet34')

    return model



In [70]:

# Test ResNet34
resnet_34 = build_resnet(input_shape=(32, 32, 3), is_50=False)
resnet_34.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        input_layer[0][0]                
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu (Activation)         (None, 16, 16, 64)   0           conv1_bn[0][0]                   
___________________________________________________________________________________________

In [71]:
# Test ResNet50
resnet_50 = build_resnet(input_shape=(32, 32, 3), is_50=True)
resnet_50.summary()


Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        input_layer[0][0]                
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu (Activation)         (None, 16, 16, 64)   0           conv1_bn[0][0]                   
___________________________________________________________________________________________

In [ ]:
# is_50 : True --> resnet_50
# is_plain :True --> no skip connection    
def build_resnet_block(input_layer, num_cnn=3, channel=64, block_num=1,is_50 = False,is_plain = False):
    # 입력 레이어
    x = input_layer
    if not is_50:
    # CNN 레이어
        for cnn_num in range(num_cnn):
            identity = x
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(3,3),
                activation='relu',
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_conv{cnn_num}'
            )(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Conv2D(
                filters=channel,
                kernel_size=(3,3),
                activation='relu',
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_1_conv{cnn_num}'
            )(x)
            if not is_plain:
                identity_channel = identity.shape.as_list()[-1]    

                if identity_channel != channel:
                    identity = keras.layers.Conv2D(channel, kernel_size=(1, 1), strides=(1, 1), padding="same")(identity)  
                # skip connection
                x = keras.layers.Add()([x,identity])   
            else:
                pass
    else :
        identity = x
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_1_conv{cnn_num}'
        )(x)
        x = keras.layers.Conv2D(
            filters=channel * 4,
            kernel_size=(1,1),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_2_conv{cnn_num}'
        )(x)
        if not is_plain:
            identity_channel = identity.shape.as_list()[-1]    

            if identity_channel != channel:
                identity = keras.layers.Conv2D(channel, kernel_size=(1, 1), strides=(1, 1), padding="same")(identity)  
            # skip connection
            x = keras.layers.Add()([x,identity])   
        else:
            pass
    #     Max Pooling 레이어
    # 마지막 블록 뒤에는 pooling을 하지 않음
    if identity.shape[1] != 1:        
        x = keras.layers.MaxPooling2D(
            pool_size=(2, 2),
            strides=2,
            name=f'block{block_num}_pooling'
        )(x)

    return x